# 15-01 어텐션 메커니즘 (Attention Mechanism) #


실습 자료 출처 : https://wikidocs.net/48948


앞서 배운 seq2seq 모델은 인코더에서 입력 시퀀스를 컨텍스트 벡터라는 
하나의 고정된 크기의 벡터 표현으로 압축하고, 

디코더는 이 컨텍스트 벡터를 통해서 출력 시퀀스를 만들어냈습니다.

하지만 이러한 RNN에 기반한 seq2seq 모델에는 크게 두 가지 문제가 있습니다.

첫째, 하나의 고정된 크기의 벡터에 모든 정보를 압축하려고 하니까 정보 손실이 발생합니다.

둘째, RNN의 고질적인 문제인 기울기 소실(vanishing gradient) 문제가 존재합니다.

### 1. 어텐션(Attention)의 아이디어


어텐션의 기본 아이디어는 디코더에서 출력 단어를 예측하는 매 시점(time step)마다, 인코더에서의 전체 입력 문장을 다시 한 번 참고한다는 점입니다. 

단, 전체 입력 문장을 전부 다 동일한 비율로 참고하는 것이 아니라, 해당 시점에서 예측해야할 단어와 연관이 있는 입력 단어 부분을 좀 더 집중(attention)해서 보게 됩니다.

### 2. 어텐션 함수(Attention Function)


어텐션 메커니즘을 언급하기 전에 컴퓨터공학의 많은 분야에서 사용되는 Key-Value로 구성되는 자료형에 대해서 잠깐 언급하겠습니다. 가령, 이 책의 주 언어로 사용되는 파이썬에도 Key-Value로 구성되는 자료형인 딕셔너리(Dict) 자료형이 존재합니다. 파이썬의 딕셔너리 자료형은 키(Key)와 값(Value)이라는 두 개의 쌍으로 구성되는데, 키를 통해서 맵핑된 값을 찾아낼 수 있다는 특징을 갖고있습니다.

In [3]:
# 파이썬의 딕셔너리 자료형을 선언
# 키(Key) : 값(value)의 형식으로 키와 값의 쌍(Pair)을 선언한다.
dict = {"2017" : "Transformer", "2018" : "BERT"}

위의 자료형에서 2017은 키에 해당되며, Transformer는 2017의 키와 맵핑되는 값에 해당됩니다. 그와 마찬가지로 2018은 키에 해당되며, BERT는 2018이라는 키와 맵핑되는 값에 해당됩니다.

In [4]:
print(dict["2017"]) #2017이라는 키에 해당되는 값을 출력


Transformer


In [6]:
print(dict["2018"])  #2018이라는 키에 해당되는 값을 출력



BERT


Key-Value 자료형에 대한 이해를 가지고 어텐션 함수에 대해서 설명해보겠습니다.

----------------------------------------------

# 15-02 바다나우 어텐션(Bahdanau Attention)


앞서 어텐션 메커니즘의 목적과 어텐션 메커니즘의 일종인 닷 프로덕트 어텐션(루옹 어텐션)의 전체적인 개요를 살펴보고, 마지막에 표를 통해 그 외에도 다양한 어텐션 메커니즘이 존재한다고 소개하였습니다. 이번에는 닷 프로덕트 어텐션보다는 조금 더 복잡하게 설계된 바다나우 어텐션 메커니즘을 이해해봅시다.

#### 1. 바다나우 어텐션 함수(Bahdanau Attention Function)


어텐션 메커니즘을 함수 Attention()으로 정의하였을 때, 바다나우 어텐션 함수의 입, 출력은 다음과 같의 정의할 수 있습니다.

Attention(Q, K, V) = Attention Value

In [10]:
# t = 어텐션 메커니즘이 수행되는 디코더 셀의 현재 시점을 의미.

# Q = Query : t-1 시점의 디코더 셀에서의 은닉 상태
# K = Keys : 모든 시점의 인코더 셀의 은닉 상태들
# V = Values : 모든 시점의 인코더 셀의 은닉 상태들

여기서는 어텐션 함수의 Query가 디코더 셀의 t 시점의 은닉 상태가 아니라 t-1 시점의 은닉 상태임을 주목합시다.



#### 2. 바다나우 어텐션(Bahdanau Attention)


1) 어텐션 스코어(Attention Score)를 구한다.

2) 소프트맥스(softmax) 함수를 통해 어텐션 분포(Attention Distribution)를 구한다.

3) 각 인코더의 어텐션 가중치와 은닉 상태를 가중합하여 어텐션 값(Attention Value)을 구한다.

4) 컨텍스트 벡터로부터를 구합니다.

---------